<a href="https://colab.research.google.com/github/ArsanyGirgis/ArsanyGirgis/blob/main/Recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Data Citation:
# F. Maxwell Harper and Joseph A. Konstan. 2015. The MovieLens Datasets: History and Context. ACM Transactions on
# Interactive Intelligent Systems (TiiS) 5, 4: 19:1–19:19.

! curl http://files.grouplens.org/datasets/movielens/ml-latest-small.zip -o ml-latest-small.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  955k  100  955k    0     0  1677k      0 --:--:-- --:--:-- --:--:-- 1675k


In [2]:
import zipfile
with zipfile.ZipFile('ml-latest-small.zip', 'r') as zip_ref:
    zip_ref.extractall('data')

In [3]:
# import the dataset
import pandas as pd
movies_df = pd.read_csv('data/ml-latest-small/movies.csv')
ratings_df = pd.read_csv('data/ml-latest-small/ratings.csv')

In [4]:
print('The dimensions of movies dataframe are:', movies_df.shape,'\nThe dimensions of ratings dataframe are:', ratings_df.shape)

# Take a look at movies_df
movies_df.head()

The dimensions of movies dataframe are: (9742, 3) 
The dimensions of ratings dataframe are: (100836, 4)


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
# Take a look at ratings_df
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [6]:
# Movie ID to movie name mapping
movie_names = movies_df.set_index('movieId')['title'].to_dict()
n_users = len(ratings_df.userId.unique())
n_items = len(ratings_df.movieId.unique())
print("Number of unique users:", n_users)
print("Number of unique movies:", n_items)
print("The full rating matrix will have:", n_users*n_items, 'elements.')
print('----------')
print("Number of ratings:", len(ratings_df))
print("Therefore: ", len(ratings_df) / (n_users*n_items) * 100, '% of the matrix is filled.')
print("We have an incredibly sparse matrix to work with here.")
print("And... as you can imagine, as the number of users and products grow, the number of elements will increase by n*2")
print("You are going to need a lot of memory to work with global scale... storing a full matrix in memory would be a challenge.")
print("One advantage here is that matrix factorization can realize the rating matrix implicitly, thus we don't need all the data")


Number of unique users: 610
Number of unique movies: 9724
The full rating matrix will have: 5931640 elements.
----------
Number of ratings: 100836
Therefore:  1.6999683055613624 % of the matrix is filled.
We have an incredibly sparse matrix to work with here.
And... as you can imagine, as the number of users and products grow, the number of elements will increase by n*2
You are going to need a lot of memory to work with global scale... storing a full matrix in memory would be a challenge.
One advantage here is that matrix factorization can realize the rating matrix implicitly, thus we don't need all the data


In [7]:
import torch
import numpy as np
from torch.autograd import Variable
from tqdm import tqdm_notebook as tqdm

class MatrixFactorization(torch.nn.Module):
    def __init__(self, n_users, n_items, n_factors=20):
        super().__init__()
        # create user embeddings
        self.user_factors = torch.nn.Embedding(n_users, n_factors) # think of this as a lookup table for the input.
        # create item embeddings
        self.item_factors = torch.nn.Embedding(n_items, n_factors) # think of this as a lookup table for the input.
        self.user_factors.weight.data.uniform_(0, 0.05)
        self.item_factors.weight.data.uniform_(0, 0.05)

    def forward(self, data):
        # matrix multiplication
        users, items = data[:,0], data[:,1]
        return (self.user_factors(users)*self.item_factors(items)).sum(1)
    # def forward(self, user, item):
    # 	# matrix multiplication
    #     return (self.user_factors(user)*self.item_factors(item)).sum(1)

    def predict(self, user, item):
        return self.forward(user, item)

In [8]:
# Creating the dataloader (necessary for PyTorch)
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader # package that helps transform your data to machine learning readiness

# Note: This isn't 'good' practice, in a MLops sense but we'll roll with this since the data is already loaded in memory.
class Loader(Dataset):
    def __init__(self):
        self.ratings = ratings_df.copy()

        # Extract all user IDs and movie IDs
        users = ratings_df.userId.unique()
        movies = ratings_df.movieId.unique()

        #--- Producing new continuous IDs for users and movies ---

        # Unique values : index
        self.userid2idx = {o:i for i,o in enumerate(users)}
        self.movieid2idx = {o:i for i,o in enumerate(movies)}

        # Obtained continuous ID for users and movies
        self.idx2userid = {i:o for o,i in self.userid2idx.items()}
        self.idx2movieid = {i:o for o,i in self.movieid2idx.items()}

        # return the id from the indexed values as noted in the lambda function down below.
        self.ratings.movieId = ratings_df.movieId.apply(lambda x: self.movieid2idx[x])
        self.ratings.userId = ratings_df.userId.apply(lambda x: self.userid2idx[x])


        self.x = self.ratings.drop(['rating', 'timestamp'], axis=1).values
        self.y = self.ratings['rating'].values
        self.x, self.y = torch.tensor(self.x), torch.tensor(self.y) # Transforms the data to tensors (ready for torch models.)

    def __getitem__(self, index):
        return (self.x[index], self.y[index])

    def __len__(self):
        return len(self.ratings)



In [9]:
num_epochs = 128
cuda = torch.cuda.is_available()

print("Is running on GPU:", cuda)

model = MatrixFactorization(n_users, n_items, n_factors=8)
print(model)
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.data)
# GPU enable if you have a GPU...
if cuda:
    model = model.cuda()

# MSE loss
loss_fn = torch.nn.MSELoss()

# ADAM optimizier
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# Train data
train_set = Loader()
train_loader = DataLoader(train_set, 128, shuffle=True)

Is running on GPU: True
MatrixFactorization(
  (user_factors): Embedding(610, 8)
  (item_factors): Embedding(9724, 8)
)
user_factors.weight tensor([[0.0452, 0.0067, 0.0155,  ..., 0.0293, 0.0493, 0.0171],
        [0.0020, 0.0045, 0.0156,  ..., 0.0021, 0.0333, 0.0110],
        [0.0391, 0.0252, 0.0382,  ..., 0.0491, 0.0047, 0.0462],
        ...,
        [0.0032, 0.0441, 0.0457,  ..., 0.0141, 0.0304, 0.0101],
        [0.0038, 0.0300, 0.0257,  ..., 0.0375, 0.0265, 0.0020],
        [0.0235, 0.0194, 0.0179,  ..., 0.0229, 0.0329, 0.0042]])
item_factors.weight tensor([[0.0435, 0.0263, 0.0242,  ..., 0.0146, 0.0373, 0.0464],
        [0.0488, 0.0299, 0.0498,  ..., 0.0416, 0.0446, 0.0309],
        [0.0148, 0.0246, 0.0222,  ..., 0.0400, 0.0416, 0.0032],
        ...,
        [0.0275, 0.0338, 0.0005,  ..., 0.0384, 0.0342, 0.0229],
        [0.0438, 0.0172, 0.0434,  ..., 0.0323, 0.0253, 0.0011],
        [0.0034, 0.0418, 0.0158,  ..., 0.0135, 0.0375, 0.0183]])


In [10]:
for it in tqdm(range(num_epochs)):
    losses = []
    for x, y in train_loader:
         if cuda:
            x, y = x.cuda(), y.cuda()
            optimizer.zero_grad()
            outputs = model(x)
            loss = loss_fn(outputs.squeeze(), y.type(torch.float32))
            losses.append(loss.item())
            loss.backward()
            optimizer.step()
    print("iter #{}".format(it), "Loss:", sum(losses) / len(losses))

<ipython-input-10-dad152416852>:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for it in tqdm(range(num_epochs)):


  0%|          | 0/128 [00:00<?, ?it/s]

iter #0 Loss: 11.058923811476848
iter #1 Loss: 4.743586157784244
iter #2 Loss: 2.476773661556583
iter #3 Loss: 1.7215746111676173
iter #4 Loss: 1.345557722872889
iter #5 Loss: 1.1283571403189965
iter #6 Loss: 0.9916233274386014
iter #7 Loss: 0.9000734374456599
iter #8 Loss: 0.8373005054323807
iter #9 Loss: 0.7921166190792461
iter #10 Loss: 0.7593334465840746
iter #11 Loss: 0.7347479461110787
iter #12 Loss: 0.7159751682414621
iter #13 Loss: 0.7015343031604883
iter #14 Loss: 0.6901948813178818
iter #15 Loss: 0.6817859854870642
iter #16 Loss: 0.6748138105279298
iter #17 Loss: 0.6697563616001061
iter #18 Loss: 0.665902410364393
iter #19 Loss: 0.6628161400632204
iter #20 Loss: 0.6607836990565212
iter #21 Loss: 0.658797782538506
iter #22 Loss: 0.6578905007939049
iter #23 Loss: 0.6569833519495079
iter #24 Loss: 0.6559799234879199
iter #25 Loss: 0.6553766271335825
iter #26 Loss: 0.654717731823776
iter #27 Loss: 0.6541720808762584
iter #28 Loss: 0.6528994291264394
iter #29 Loss: 0.6525553975840

In [11]:

# By training the model, we will have tuned latent factors for movies and users.
c = 0
uw = 0
iw = 0
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.data)
        if c == 0:
          uw = param.data
          c +=1
        else:
          iw = param.data
        #print('param_data', param_data)~

user_factors.weight tensor([[ 1.6874,  1.1514,  1.6076,  ...,  1.4993,  0.6057,  1.2229],
        [ 1.5567, -0.0640,  0.9257,  ...,  0.6028,  1.2154,  1.3924],
        [ 2.2755, -2.8848, -0.4556,  ...,  1.9844,  0.7597,  0.5994],
        ...,
        [ 1.2689,  1.1157,  0.8436,  ...,  2.0532,  1.8526,  0.3147],
        [ 1.0776,  0.5122,  0.9050,  ...,  0.5536,  1.7093,  1.1632],
        [ 0.6987,  0.7464,  0.3451,  ...,  1.4469,  1.1507,  0.8100]],
       device='cuda:0')
item_factors.weight tensor([[ 0.4306,  0.5368,  0.5735,  ...,  0.1199,  0.3889,  0.5371],
        [ 0.2494, -0.0422,  0.6371,  ...,  0.0472,  0.2594,  0.5251],
        [ 0.1894,  0.4474,  0.4949,  ...,  0.5335,  0.5386,  0.3549],
        ...,
        [ 0.3567,  0.3655,  0.3059,  ...,  0.3692,  0.3655,  0.3540],
        [ 0.4267,  0.4009,  0.3975,  ...,  0.4150,  0.4088,  0.3840],
        [ 0.3971,  0.4336,  0.3899,  ...,  0.4057,  0.4303,  0.4112]],
       device='cuda:0')


In [14]:
trained_movie_embeddings = model.item_factors.weight.data.cpu().numpy()


trained_movie_embeddings # unique movie factor weights

array([[ 0.43057418,  0.536815  ,  0.5735027 , ...,  0.11993974,
         0.38885432,  0.5370783 ],
       [ 0.24942316, -0.04218983,  0.6371368 , ...,  0.04720813,
         0.25942984,  0.5250747 ],
       [ 0.18935467,  0.4473871 ,  0.4948607 , ...,  0.53354216,
         0.5385552 ,  0.354939  ],
       ...,
       [ 0.35674724,  0.3655468 ,  0.30588657, ...,  0.3691985 ,
         0.3654833 ,  0.3540342 ],
       [ 0.426733  ,  0.40089843,  0.39751297, ...,  0.41495517,
         0.40880612,  0.38395286],
       [ 0.3971082 ,  0.43361026,  0.38985908, ...,  0.4057476 ,
         0.43033868,  0.4111826 ]], dtype=float32)

In [13]:
from sklearn.cluster import KMeans
# Fit the clusters based on the movie weights
kmeans = KMeans(n_clusters=10, random_state=0).fit(trained_movie_embeddings)


'''It can be seen here that the movies that are in the same cluster tend to have
similar genres. Also note that the algorithm is unfamiliar with the movie name
and only obtained the relationships by looking at the numbers representing how
users have responded to the movie selections.'''
for cluster in range(10):
  print("Cluster #{}".format(cluster))
  movs = []
  for movidx in np.where(kmeans.labels_ == cluster)[0]:
    movid = train_set.idx2movieid[movidx]
    rat_count = ratings_df.loc[ratings_df['movieId']==movid].count()[0]
    movs.append((movie_names[movid], rat_count))
  for mov in sorted(movs, key=lambda tup: tup[1], reverse=True)[:10]:
    print("\t", mov[0])

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Cluster #0
	 Forrest Gump (1994)
	 Shawshank Redemption, The (1994)
	 Silence of the Lambs, The (1991)
	 Matrix, The (1999)
	 Star Wars: Episode IV - A New Hope (1977)
	 Terminator 2: Judgment Day (1991)
	 Toy Story (1995)
	 Star Wars: Episode V - The Empire Strikes Back (1980)
	 Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)
	 Fugitive, The (1993)
Cluster #1
	 GoldenEye (1995)
	 Twister (1996)
	 Net, The (1995)
	 Clear and Present Danger (1994)
	 Firm, The (1993)
	 Starship Troopers (1997)
	 Dark Knight Rises, The (2012)
	 Robin Hood: Men in Tights (1993)
	 Avengers, The (2012)
	 Eraser (1996)
Cluster #2
	 Ace Ventura: Pet Detective (1994)
	 Mask, The (1994)
	 Batman Forever (1995)
	 Pretty Woman (1990)
	 Dumb & Dumber (Dumb and Dumber) (1994)
	 Waterworld (1995)
	 Harry Potter and the Chamber of Secrets (2002)
	 Matrix Reloaded, The (2003)
	 Armageddon (1998)
	 Mummy, The (1999)
Cluster #3
	 Coneheads (1993)
	 Batman & Robin (1997)
	 Godzilla (1998)
	 